# Generation of preview

The following function will generate a 2-dim preview of the maze using `IPython.display` and HTML.

In [ ]:
from IPython.display import display, HTML

def laby2d(laby, current=[0,0,0], details=0, path={}):
    def trace_path(i,j):
      if (i,j) in path:
        d = {"s": "⬇️", "n": "⬆️", "e": "➡️", "w": "⬅️"}
        return d[path[(i,j)]]
      else:
        return ''
    style = """
    <style>
    .container {
      display: inline-block;
      border:5px solid black;
      user-select:none;
    }

    .row-container {
      display: flex;
    }

    .cell {
      color: black;
      margin:0;
      background: white;
      /*border:1px rgba(0,0,0,0.3) dotted;*/
      height: 45px;
      width: 45px;
      display:flex;
      align-items:center;
      justify-content:center;
    }

    .wall-v {
    	background:white;
        width:5px;
        margin:0;
    }

    .wall-v.enable{
    background:black
    }

    .wall-h {
    	background:white;
        width:50px;
        /*border-left:3px solid black;
        border-right:3px solid black;*/
        height:5px;
    }

	.wall-h.enable{
    background:black;
    border-left: 0px solid black;
    }

    .cell.blue{
      background:blue;
    }

    .cell.yellow{
      background:yellow;
    }

    .row-container:first-child > .cell:first-child{
      border-top:solid 15px green;
      height:30px;
    }

    .row-container:last-child > .cell:last-child{
      border-bottom:solid 15px red;
      height:30px;
    }

    .cell:last-child{
        width:50px;
    }


  </style>
    """
    dims = laby[0].shape
    html = "<div class='container'>"
    for i in range(dims[0]):
      elem = "\n  <div class='row-container'>\n"
      row_walls="\n \n<div class='row-container'>\n"
      for j in range(dims[1]):
        elem+=f"   <div class='cell {['','','yellow','blue'][laby[0][i][j]]} {['','blue'][[i,j]==current]}'>{['', (i,j)][details]} {trace_path(i,j)}</div>"
        elem+=' '+f"\n  <div class=\"wall-v {['','enable'][int(laby[0][i][j]==1)]}\"></div>\n"
      elem+=f"  <div class='cell {['','','yellow','blue'][laby[0][i][j]]} {['','blue'][[i,j+1]==current]}'>{['',(i,j+1)][details]} {trace_path(i,j+1)}</div>\n  </div>\n"
      if i!=dims[0]-1:
        for wall_h in laby[1][i]:
          row_walls+=f"<div class='wall-h {['','enable'][int(wall_h==1)]}'></div>\n"
        row_walls+='</div>\n \n'
        elem+=row_walls
      html+=elem
    html+='</div>'
    display(HTML(style+html))
    # print(html)

# Generation of maze

The following function, with the superimposition of 4 matrices of the same dimensions will return a list of matrices, defining the maze (each matrice is for a certain-direction of wall, respectively west, east, north and south).

In [ ]:
import numpy as np

def maze_full_walls(x,y):
  # order : ['west', 'east', 'south', 'north']
  laby = [np.ones((x, y-1), dtype=np.int32), np.ones((x-1, y), dtype=np.int32)]
  return laby
laby2d(maze_full_walls(5,5))

In [ ]:
class Stack:
    def __init__(self):
        self.items = []

    def is_empty(self):
        return len(self.items) == 0

    def push(self, item):
        self.items.append(item)
        return item

    def pop(self):
        if not self.is_empty():
            return self.items.pop()
        else:
            return None

    def peek(self):
        if not self.is_empty():
            return self.items[-1]
        else:
            return None

    def size(self):
        return len(self.items)

    def __repr__(self):
        return self.items()

def get_neigh(dims, laby, ignore_wall=0):
  neigh = []
  laby = laby
  # check if there is adjacent cells (right)
  if dims[2]!=0:
    if not laby[0][dims[1]][dims[2]-1] or ignore_wall:
      neigh.append([0, dims[1], dims[2]-1])
  if dims[2]!=laby[0].shape[1]:
    if not laby[0][dims[1]][dims[2]] or ignore_wall:
      neigh.append([0, dims[1], dims[2]+1])

  # handle top / bottom neighboors
  if dims[1]!=0:
    if not laby[1][dims[1]-1][dims[2]] or ignore_wall:
      neigh.append([1, dims[1]-1, dims[2]])
  if dims[1]!=laby[0].shape[1]:
    if not laby[1][dims[0]][dims[1]] or ignore_wall:
      neigh.append([1, dims[1]+1, dims[2]])

  return neigh


In [ ]:
from random import choice, randint

def generate_maze(x, y):
    laby = maze_full_walls(x, y)
    def remove_wall(z, x1,y1, x2, y2):
      if z==1:
        laby[z][min(x1,x2)][y1]=0
      else:
        laby[z][x1][min(y1, y2)]=0
    rel = Stack()
    x = rel.push([0,randint(x-1, y-1),randint(x-1, y-1)])
    visited = [[x[0],x[1]]]
    while not rel.is_empty():
      neigh = [n for n in get_neigh(x, laby, ignore_wall=1) if n[1:] not in visited]
      while len(neigh)==0:
        x = rel.pop()
        if rel.is_empty():
          break
        neigh = [n for n in get_neigh(x, laby, ignore_wall=1) if n[1:] not in visited]
      if rel.is_empty():
        break
      x0 = x.copy()
      x = choice(neigh)
      remove_wall(x[0], x[1], x[2], x0[1], x0[2])
      x = rel.push(x)
      visited.append(x[1:])

    return laby


# Solve maze

Solve maze (by adding arrows to show the resolution path) with the function `solve_maze`

In [ ]:
def solve_maze(maze):
  path = {}
  res = Stack()
  x = [0,0,0]
  end = [maze[0].shape[0]-1 for i in range(2)]
  print(path)
  i = 0
  visited = [x[1:]]
  while x!=end and i<2000:
    i+=1
    res.push(x)
    neigh = [n for n in get_neigh(x, maze) if n[1:] not in visited]
    if len(neigh)==0:
      while len(neigh)==0:
        i+=1
        x = res.pop()
        neigh = [n for n in get_neigh(x, maze) if n[1:] not in visited]
    x0 = tuple(x.copy())
    x = choice(neigh)
    if x[0]==1:
      if x[1]>x0[1]:
        path[x0[1:]]='e'
      else:
        path[x0[1:]]='w'
    else:
      if x[2]>x0[2]:
        path[x0[1:]]='s'
      else:
        path[x0[1:]]='n'
    # laby2d(maze, path=path, current=x[1:])

  return path


maze = generate_maze(10,10)
laby2d(maze, path=solve_maze(maze))
laby2d(maze, details=1)

{}
